In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
#Previous without latlong
#data=pd.read_csv("D:/programming-no-gdrive/air-pollution/data-formatted/03.03.19-restrict-hours/all_sites-2000.csv")
#Current
data=pd.read_csv("D:/programming-no-gdrive/air-pollution/data-formatted/03.03.19-restrict-hours/all_sites-2000-wlatlong.csv")
data = data.drop(['AQS_Code'], axis =1)

This data has no nulls in it. Currently testing on just 2000 data. My idea is that we should do one interpolation with 2000 and one with 2017. 

In [5]:
data['month'].value_counts()

12.0    19144
10.0    18933
8.0     18694
11.0    18097
9.0     17813
7.0     17096
3.0     16353
5.0     16257
4.0     15996
6.0     15832
2.0     14833
Name: month, dtype: int64

Last time I made a note that January had far less data than other months so I would drop it. Here it is already not represented at all.

In [5]:
X_train, X_test, y_train, y_test=train_test_split(data.drop(['o3'],axis = 1) , data[['o3']], random_state = 1)

<h2>Normalize below for neural networks

In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

<h2> Convert Y to np arrays for neural networks

In [9]:
y_train = np.array(y_train)
y_test = np.array(y_test)

<h2>Try normalization on Y

In [21]:
y_train = scaler.fit_transform(y_train)
y_test = scaler.fit_transform(y_test)

<h2>First Linear Regression</h2>

In [33]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X_train,y_train)
print(reg.score(X_test,y_test))

0.04433974365258364


<h2>Random Forest</h2>

First try without normalization

In [30]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 10, random_state = 0, verbose = 1)
# Train the model on training data
rf.fit(X_train, y_train)

C:\Users\nicholas\Anaconda3\envs\deep-learning\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   13.1s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=1, warm_start=False)

In [31]:
print(rf.score(X_test, y_test))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.42803537086281207


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s finished


<h2>Below is older

In [21]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf1000 = RandomForestRegressor(n_estimators = 1000, random_state = 0, verbose = 1)
# Train the model on training data
rf1000.fit(X_train, y_train)

C:\Users\nicholas\Anaconda3\envs\deep-learning\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed: 20.0min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=0, verbose=1, warm_start=False)

In [22]:
print(rf1000.score(X_test, y_test))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.47667216690489556


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   20.9s finished


In [23]:
from sklearn.externals import joblib
joblib_file = "hrly_rf_10.pkl"  
joblib.dump(rf, joblib_file)

['hrly_rf_10.pkl']

<h2>Artificial Neural Network

In [12]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [25]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=10, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
# Compile model
model.compile(loss='mean_absolute_error', optimizer ='adam', metrics=['mae'])

In [26]:
# Fit the model
model.fit(X_train, y_train, epochs=50, batch_size=50, verbose = True)

Epoch 1/50
141786/141786 [==============================] - 3s 23us/step - loss: 0.0271 - mean_absolute_error: 0.0271
Epoch 2/50
141786/141786 [==============================] - 3s 18us/step - loss: 0.0194 - mean_absolute_error: 0.0194
Epoch 3/50
141786/141786 [==============================] - 3s 18us/step - loss: 0.0191 - mean_absolute_error: 0.0191
Epoch 4/50
141786/141786 [==============================] - 3s 19us/step - loss: 0.0186 - mean_absolute_error: 0.0186
Epoch 5/50
141786/141786 [==============================] - 3s 19us/step - loss: 0.0184 - mean_absolute_error: 0.0184
Epoch 6/50
141786/141786 [==============================] - 3s 20us/step - loss: 0.0183 - mean_absolute_error: 0.0183
Epoch 7/50
141786/141786 [==============================] - 3s 19us/step - loss: 0.0181 - mean_absolute_error: 0.0181
Epoch 8/50
141786/141786 [==============================] - 3s 19us/step - loss: 0.0180 - mean_absolute_error: 0.0180
Epoch 9/50
141786/141786 [==============================

In [27]:
model.evaluate(X_test, y_test)

47262/47262 [==============================] - 1s 15us/step


[0.030036334211324334, 0.030036334211324334]

In [28]:
model.save("D:/programming-no-gdrive/air-pollution/models/hrly_2000_ann.h5")

<h2> ANN Modeled after Carroll's LSTM's parameters

In [29]:
from keras import backend as K
def r2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [30]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=10, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
# Compile model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[r2, 'mae'])

In [31]:
model.fit(X_train, y_train, epochs=50, batch_size=50, verbose = True)

Epoch 1/50
141786/141786 [==============================] - 3s 22us/step - loss: 0.0026 - r2: -0.3229 - mean_absolute_error: 0.0263
Epoch 2/50
141786/141786 [==============================] - 3s 19us/step - loss: 0.0022 - r2: 0.0804 - mean_absolute_error: 0.0220
Epoch 3/50
141786/141786 [==============================] - 3s 20us/step - loss: 0.0021 - r2: 0.1127 - mean_absolute_error: 0.0213
Epoch 4/50
141786/141786 [==============================] - 3s 19us/step - loss: 0.0021 - r2: 0.1353 - mean_absolute_error: 0.0210
Epoch 5/50
141786/141786 [==============================] - 3s 21us/step - loss: 0.0021 - r2: 0.1396 - mean_absolute_error: 0.0209
Epoch 6/50
141786/141786 [==============================] - 3s 20us/step - loss: 0.0021 - r2: 0.1406 - mean_absolute_error: 0.0208
Epoch 7/50
141786/141786 [==============================] - 3s 20us/step - loss: 0.0020 - r2: 0.1369 - mean_absolute_error: 0.0208
Epoch 8/50
141786/141786 [==============================] - 3s 19us/step - loss: 0

In [32]:
model.evaluate(X_test, y_test)

47262/47262 [==============================] - 1s 15us/step


[0.006582647016144493, -6.734042973268361, 0.042732740269968275]